In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense


Using TensorFlow backend.


In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
def loadTrain():
    root = ''
    train_df = pd.read_csv(root + 'train.csv')
    train_df["timestamp"] = pd.to_datetime(train_df["timestamp"], format='%Y-%m-%d %H:%M:%S')
    train_df = reduce_mem_usage(train_df)

    weather_train_df = pd.read_csv(root + 'weather_train.csv')
    weather_train_df["timestamp"] = pd.to_datetime(weather_train_df["timestamp"], format='%Y-%m-%d %H:%M:%S')
    weather_train_df = reduce_mem_usage(weather_train_df)
    # test_df = pd.read_csv(root + 'test.csv')
    # test_df["hour"] = pd.to_datetime(test_df["timestamp"], format='%Y-%m-%d %H:%M:%S').dt.hour

    # weather_test_df = pd.read_csv(root + 'weather_test.csv')
    building_meta_df = pd.read_csv(root + 'building_metadata.csv')
    building_meta_df = reduce_mem_usage(building_meta_df)
    return (train_df,weather_train_df,building_meta_df)



In [4]:
def MergeT():
    train_df,weather_train_df,building_meta_df = loadTrain()
    b_data = pd.merge(train_df, building_meta_df, on='building_id')
    b_0 = b_data
    b = b_0.drop(columns=['primary_use', 'year_built', 'floor_count'])
    #b = b_0.drop(columns=['year_built', 'floor_count'])
    b = pd.merge(b, weather_train_df, on=['site_id','timestamp'])
    b['timestamp'] = pd.to_datetime(b["timestamp"], format='%Y-%m-%d %H:%M:%S')
    print("records:", len(b))

    b = b.drop(columns=['cloud_coverage', 'precip_depth_1_hr', 'sea_level_pressure', 
                'wind_direction', 'wind_speed'])

    b = b.dropna()
    print("records after dropping NaN:", len(b))

    b['Y'] = b['meter_reading'] / b['square_feet']*100
    b['workhour'] = np.abs(b['timestamp'].dt.hour - 12) < 5.5
    b['workday'] = b['timestamp'].dt.weekday < 5
    b = b.drop(columns = ["site_id","square_feet","timestamp","meter_reading"])
    print("rows in dataset:", len(b))
    b = b.astype({'building_id':int, 'workhour': int, 'workday': int})
    b = reduce_mem_usage(b)
    return b

In [5]:
def LoadTest():
    root = ''
    test_df = pd.read_csv(root + 'test.csv')
    test_df["timestamp"] = pd.to_datetime(test_df["timestamp"], format='%Y-%m-%d %H:%M:%S')
    test_df = reduce_mem_usage(test_df)

    weather_test_df = pd.read_csv(root + 'weather_test.csv')
    weather_test_df["timestamp"] = pd.to_datetime(weather_test_df["timestamp"], format='%Y-%m-%d %H:%M:%S')
    weather_test_df = reduce_mem_usage(weather_test_df)
    # test_df = pd.read_csv(root + 'test.csv')
    # test_df["hour"] = pd.to_datetime(test_df["timestamp"], format='%Y-%m-%d %H:%M:%S').dt.hour

    # weather_test_df = pd.read_csv(root + 'weather_test.csv')
    building_meta_df = pd.read_csv(root + 'building_metadata.csv')
    building_meta_df = reduce_mem_usage(building_meta_df)
    return (test_df,weather_test_df,building_meta_df)

def MergeTest():
    (test_df,weather_test_df,building_meta_df) = LoadTest()
    b_data = pd.merge(test_df, building_meta_df, on='building_id')
    b_0 = b_data
    print("records after merge building",len(b_0))
    b = b_0.drop(columns=['primary_use', 'year_built', 'floor_count'])
    b = pd.merge(b, weather_test_df, on=['site_id','timestamp'],how="left")
    b['timestamp'] = pd.to_datetime(b["timestamp"], format='%Y-%m-%d %H:%M:%S')
    print("records:", len(b))

    b = b.drop(columns=['cloud_coverage', 'precip_depth_1_hr', 'sea_level_pressure', 
                'wind_direction', 'wind_speed'])

    #b = b.dropna()
    print("records after dropping NaN:", len(b))

    b['workhour'] = np.abs(b['timestamp'].dt.hour - 12) < 5.5
    b['workday'] = b['timestamp'].dt.weekday < 5
    b = b.drop(columns = ["site_id","timestamp"])
    print("rows in dataset:", len(b))
    b = b.astype({'building_id':int, 'workhour': int, 'workday': int})
    b = reduce_mem_usage(b)
    return b

In [6]:
test_df = MergeTest()
test_df

Mem. usage decreased to 596.49 Mb (53.1% reduction)
Mem. usage decreased to  6.08 Mb (68.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)
records after merge building 41697600
records: 41697600
records after dropping NaN: 41697600
rows in dataset: 41697600
Mem. usage decreased to 994.15 Mb (24.2% reduction)


,row_id,building_id,meter,square_feet,air_temperature,dew_temperature,workhour,workday
0,0,0,0,7432,17.796875,11.703125,0,0
1,129,0,0,7432,17.796875,12.796875,0,0
2,258,0,0,7432,16.093750,12.796875,0,0
3,387,0,0,7432,17.203125,13.296875,0,0
4,516,0,0,7432,16.703125,13.296875,0,0
...,...,...,...,...,...,...,...,...
41697595,41692949,1448,0,92271,NaN,NaN,1,1
41697596,41693199,1448,0,92271,NaN,NaN,1,1
41697597,41697099,1448,0,92271,NaN,NaN,0,1
41697598,41697349,1448,0,92271,NaN,NaN,0,1


In [7]:
train_df,weather_train_df,building_meta_df = loadTrain()

Mem. usage decreased to 289.19 Mb (53.1% reduction)
Mem. usage decreased to  3.07 Mb (68.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)


In [8]:
building_meta_df


,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN
...,...,...,...,...,...,...
1444,15,1444,Entertainment/public assembly,19619,1914.0,NaN
1445,15,1445,Education,4298,NaN,NaN
1446,15,1446,Entertainment/public assembly,11265,1997.0,NaN
1447,15,1447,Lodging/residential,29775,2001.0,NaN


In [9]:
b = MergeT()

Mem. usage decreased to 289.19 Mb (53.1% reduction)
Mem. usage decreased to  3.07 Mb (68.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)
records: 20125605
records after dropping NaN: 20115960
rows in dataset: 20115960
Mem. usage decreased to 364.50 Mb (34.5% reduction)


In [10]:
b

,building_id,meter,air_temperature,dew_temperature,Y,workhour,workday
0,0,0,25.000000,20.000000,0.000000,0,1
1,1,0,25.000000,20.000000,0.000000,0,1
2,2,0,25.000000,20.000000,0.000000,0,1
3,3,0,25.000000,20.000000,0.000000,0,1
4,4,0,25.000000,20.000000,0.000000,0,1
...,...,...,...,...,...,...,...
20125600,1400,1,1.700195,1.700195,0.072632,1,1
20125601,1400,1,2.800781,2.199219,0.118530,1,1
20125602,1400,1,5.601562,4.398438,0.152832,1,1
20125603,1400,1,11.101562,4.398438,0.114441,1,1


In [ ]:
resu = pd.DataFrame(columns = ["row_id","meter_reading"])
subres = pd.DataFrame(columns = ["row_id","meter_reading"])
for b_id in range(1449):
    for m in range(4):
        print("building",b_id)
        bld = b[b.building_id==b_id]
        bld = bld[bld.meter==m]
        bld = bld[bld.Y > 1e-10]
        bld = bld.drop(columns = ["building_id"])
        bld = bld.drop(columns = ["meter"])
        bld = bld.dropna()
        Y = bld.values[:,2]
        b_x = bld.drop(columns=['Y'])
        X = b_x.values / np.array([50, 10, 1, 1])
        if len(X)<128:
            p = lambda x:[[0.1]]
        else:
            model = Sequential()
            model.add(Dense(6, input_dim=4, activation="relu"))
            model.add(Dense(1, activation="relu"))
            adam = keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)
            model.compile(loss="mean_squared_error", optimizer=adam, metrics=['accuracy'])
            model.fit(X, Y, epochs=50, batch_size=128, verbose = False)
            p = model.predict
            
        t = test_df[test_df.building_id == b_id]
        t = t[t.meter == m]
        t = t.drop(columns = ["meter","building_id"])
        l = []
        for i,row in t.iterrows():
            row_id, square_feet, air_temperature, dew_temperature, workhour, workday = row
            
            persqf = p(np.array([[air_temperature, dew_temperature, workhour, workday]]))[0][0]
            meter_reading = persqf * square_feet/100
            l.append(meter_reading)
        t["meter_reading"] = l
        t = t.drop(columns=["square_feet", "air_temperature", "dew_temperature", "workhour", "workday"])
        #resu = resu.append(t)
        subres = subres.append(t)
    if(b_id%50==48):
        print("saved")
        subres.to_csv("res/"+str(b_id)+".csv",index=False)
        subres = pd.DataFrame(columns = ["row_id","meter_reading"])
subres.to_csv("res/resufin.csv",index=False)

building 0
building 0
building 0
building 0
building 1
building 1
building 1
building 1
building 2
building 2
building 2
building 2
building 3
building 3
building 3
building 3
building 4
building 4
building 4
building 4
building 5
building 5
building 5
building 5
building 6
building 6
building 6
building 6
building 7
building 7
building 7
building 7
building 8
building 8
building 8
building 8
building 9
building 9
building 9
building 9
building 10
building 10
building 10
building 10
building 11
building 11
building 11
building 11
building 12
building 12
building 12
building 12
building 13
building 13
building 13
building 13
building 14
building 14
building 14
building 14
building 15
building 15
building 15
building 15
building 16
building 16
building 16
building 16
building 17
building 17
building 17
building 17
building 18
building 18
building 18
building 18
building 19
building 19
building 19
building 19
building 20
building 20
building 20
building 20
building 21
building 21
building

building 165
building 166
building 166
building 166
building 166
building 167
building 167
building 167
building 167
building 168
building 168
building 168
building 168
building 169
building 169
building 169
building 169
building 170
building 170
building 170
building 170
building 171
building 171
building 171
building 171
building 172
building 172
building 172
building 172
building 173
building 173
building 173
building 173
building 174
building 174
building 174
building 174
building 175
building 175
building 175
building 175
building 176
building 176
building 176
building 176
building 177
building 177
building 177
building 177
building 178
building 178
building 178
building 178
building 179
building 179
building 179
building 179
building 180
building 180
building 180
building 180
building 181
building 181
building 181
building 181
building 182
building 182
building 182
building 182
building 183
building 183
building 183
building 183
building 184
building 184
building 184
building 184

building 323
building 323
building 323
building 324
building 324
building 324
building 324
building 325
building 325
building 325
building 325
building 326
building 326
building 326
building 326
building 327
building 327
building 327
building 327
building 328
building 328
building 328
building 328
building 329
building 329
building 329
building 329
building 330
building 330
building 330
building 330
building 331
building 331
building 331
building 331
building 332
building 332
building 332
building 332
building 333
building 333
building 333
building 333
building 334
building 334
building 334
building 334
building 335
building 335
building 335
building 335
building 336
building 336
building 336
building 336
building 337
building 337
building 337
building 337
building 338
building 338
building 338
building 338
building 339
building 339
building 339
building 339
building 340
building 340
building 340
building 340
building 341
building 341
building 341
building 341
building 342
building 342

building 480
building 481
building 481
building 481
building 481
building 482
building 482
building 482
building 482
building 483
building 483
building 483
building 483
building 484
building 484
building 484
building 484
building 485
building 485
building 485
building 485
building 486
building 486
building 486
building 486
building 487
building 487
building 487
building 487
building 488
building 488
building 488
building 488
building 489
building 489
building 489
building 489
building 490
building 490
building 490
building 490
building 491
building 491
building 491
building 491
building 492
building 492
building 492
building 492
building 493
building 493
building 493
building 493
building 494
building 494
building 494
building 494
building 495
building 495
building 495
building 495
building 496
building 496
building 496
building 496
building 497
building 497
building 497
building 497
building 498
building 498
building 498
building 498
saved
building 499
building 499
building 499
buildi

building 638
building 638
building 638
building 638
building 639
building 639
building 639
building 639
building 640
building 640
building 640
building 640
building 641
building 641
building 641
building 641
building 642
building 642
building 642
building 642
building 643
building 643
building 643
building 643
building 644
building 644
building 644
building 644
building 645
building 645
building 645
building 645
building 646
building 646
building 646
building 646
building 647
building 647
building 647
building 647
building 648
building 648
building 648
building 648
saved
building 649
building 649
building 649
building 649
building 650
building 650
building 650
building 650
building 651
building 651
building 651
building 651
building 652
building 652
building 652
building 652
building 653
building 653
building 653
building 653
building 654
building 654
building 654
building 654
building 655
building 655
building 655
building 655
building 656
building 656
building 656
building 656
buildi

In [ ]:
rrr = pd.read_csv("resufin.csv")



In [ ]:
print(len(rrr['row_id'].unique()) - len(rrr.index))
print(len(rrr))

In [ ]:
rrr = rrr.fillna(350)
print(len(rrr['row_id'].unique()) - len(rrr.index))

In [ ]:
rrr.to_csv("resufin1.csv",index=False)

In [ ]:
rrr